In [43]:
import numpy as np
import pandas as pd

#Importing necessary libraries

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import  DecisionTreeClassifier

In [45]:
import seaborn as sns

titanic_c = sns.load_dataset('titanic')

# Save the dataset to a CSV file
titanic_c.to_csv('titanic.csv', index=False)


In [46]:
df = titanic_c

In [47]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [48]:
df.drop(columns=['adult_male','alone', 'deck','who','alive','class','embark_town'],inplace=True)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['survived']),df['survived'],test_size=0.2, random_state=42)

In [50]:
X_train.head()

,pclass,sex,age,sibsp,parch,fare,embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [51]:
y_train.sample(5)

,survived
730,1
221,0
249,0
737,1
466,0


#steps in pipeline

In [52]:
#imputation transformer

trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(),[2]),
    ('imputer_embark', SimpleImputer(strategy='most_frequent'),[6])
], remainder= 'passthrough')

In [53]:
#one hot encodeing

trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[1,6])
], remainder= 'passthrough')

In [54]:
#scaling

trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))
])

In [55]:
#feature Selection

trf4 = SelectKBest(score_func=chi2, k=5)

In [56]:
#train the model

trf5 = DecisionTreeClassifier()

In [57]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

In [58]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('imputer_embark',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x7fd79f0f5090>)),
                ('trf5', DecisionTreeClassifier())])

#Exploring a Pipe

In [59]:
#Explore the pipe



pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('imputer_embark',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=5, score_func=<function chi2 at 0x7fd79f0f5090>),
 'trf5': DecisionTreeClassifier()}

In [60]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [61]:
y_pred = pipe.predict(X_test)

In [62]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.6256983240223464

In [63]:
### Cross Validation Using Pipeline
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

#Hyper Parameter Tuning

In [64]:
# GridSearch using Pipeline
# hyperparamater tuning

params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}



In [65]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params,  cv=5, scoring ='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('imputer_embark',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=5,
                                                    score_func=<function chi2 at 0x7fd79f0f5090>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [66]:
grid.best_score_

0.6391214419383433

In [67]:
grid.best_params_

{'trf5__max_depth': 2}

In [68]:
import pickle

pickle.dump(pipe,open('pipe.pkl','wb'))

#Production Code

In [71]:
# Import necessary libraries
import pickle
import numpy as np
import pandas as pd

# Load the pre-trained pipeline
pipe = pickle.load(open('pipe.pkl', 'rb'))

# Assume user input
test_input2 = np.array([2, 'Male', 31.0, 0, 0, 10.5, 'S'], dtype=object).reshape(1, 7)
columns = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']

# Convert test_input2 to a DataFrame with column names
test_input2_df = pd.DataFrame(test_input2, columns=columns)

# Use test_input2_df for prediction to avoid the warning
prediction = pipe.predict(test_input2_df)
print(prediction)


[0]
